### 1. Install - Import Libraries

Install required python libraries.

Import python libraries.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import pandas as pd
import numpy as np

from nltk.translate.bleu_score import sentence_bleu
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.jit import script, trace
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

from datatile.summary.df import DataFrameSummary
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
%matplotlib inline
plt.close()

Set pytorch device to GPU cuda if present, otherwise set to CPU.

In [ ]:
 # set device to GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 2. Import - Review Data

#### 2.1 Download corpus and set variables

Download corpus from: https://www.kaggle.com/datasets/rtatman/ubuntu-dialogue-corpus/download?datasetVersionNumber=2 and expand compressed archive in to the notebook's root folder.

Save variables for data folder, corpus name, and print first 10 lines to review data.

In [ ]:
data = "./archive/" # data root folder
corpus_name = "Ubuntu-dialogue-corpus/" # corpus subfolder
corpus = os.path.join(data, corpus_name) # join
raw_data = "dialogueText.csv" # Existing raw data file
formatted_data = "formatted_dialogue_lines.txt" # Will be creaated later on

# print first 10 lines
def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, raw_data))

#### 2.2 Load data - initial review and clean up

Load *'dialogueText.csv'* to pandas series object and print first 10 lines for review.

In [ ]:
df = pd.read_csv(corpus + raw_data)
df.head(10)

Drop unused columns to save processing time and remove *'.tsv'* from **dialogueID**.
Print first 10 lines for review.

In [ ]:
df.drop(['folder', 'date'], axis=1, inplace=True)
df['dialogueID'] = df['dialogueID'].str.replace(r'.tsv', '', regex=True)
df.head(10)

Use DataFrameSummary method to review data column statistics.

In [ ]:
DataFrameSummary(df).columns_stats

Drop null values from 'text' and 'from' columns. The 'to' column has a lot of missing values but this is expected since all first comments are adressed to null. Review column statistics again.

In [ ]:
df.dropna(subset=['text', 'from'], inplace=True)
DataFrameSummary(df).columns_stats

### 3. Data Transformation & Analysis

#### 3.1 Transform data to Input Output pairs

Convert dataframe to dictionary using dataframe's index as key and row values as nested dictionary. Print example.

In [ ]:
#Convert dataframe to dictionary using dataframes index as key and row values as nested dictionary.
conv_dict = df.to_dict('index')

# print example
conv_dict[0]

Define helper functions for combining consecutive text from the same person to one input and for extracting input-ouput pairs from conversations.

In [ ]:
# Combine consecutive text from same person to one sentence
def combine_text(dictionary):
    for key in reversed(dictionary):  # loop on reversed dictionary
        if key+1 in dictionary and dictionary[key]['from'] == dictionary[key+1]['from']:  # if current iteration and next are from same user
            dictionary[key+1]['text'] = str(dictionary[key]['text']) + " " + str(dictionary[key+1]['text'])  # get text from current and append to next iteration text
            dictionary[key]['from'] = "del"  # set "from" value to "del"

    for key, value in dict(dictionary).items(): # loop over dictionary
        if value['from'] == "del": # if "from" value is "del"
                del dictionary[key] # delete line
                
    return dictionary   

# Extracts input output pairs from conversations
def extract_pairs(conversations):
    qa_pairs = [] # initiate pairs dictionary
        # Iterate over all the lines of the conversation
    for key in conversations:
        if key+1 in conversations and conversations[key]['dialogueID'] == conversations[key+1]['dialogueID']: # if not beyond index and dialogue ID equals next iterations dialogue ID
            inputLine = str(conversations[key]['text']).strip() # set input text as current iteration
            targetLine = str(conversations[key+1]['text']).strip() # set output text as next iteration
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine: 
                qa_pairs.append([inputLine, targetLine]) # append pait to pairs dictionary
    return qa_pairs                         

Set deliniter and new file path, run **combine_text** and **extract_pairs** functions, and write results to *"formatted_dialogue_lines.txt"* file. Print 10 lines to review data.

In [ ]:
'''
Apply combine text and extract pairs functions that complete data transformation
'''
# Set delimiter
delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Define path to new file
datafile = os.path.join(corpus, formatted_data)

print("\nProcessing corpus into lines and conversation pairs...")
dictionary = combine_text(conv_dict)
pairs = extract_pairs(dictionary)

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in pairs:
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(os.path.join(corpus, formatted_data))         

#### 3.2 Data Analysis

Load data from *"formatted_dialogue_lines.txt"* file to pandas object for data analysis. Set dataframe labels, and print first 10 lines.

In [ ]:
labels  = ["input", "output"]
df = pd.read_csv(corpus + formatted_data, sep="\t", header=None, names=labels )
df.head(10)

Plot input and output most used words

In [ ]:
for label in labels:  # loop over labels
    df_words = " ".join(word for word in df[label])  # create list of words
    wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(df_words)  # call WordCloud to generate
    custom_title = label.capitalize() + " Data Word Cloud"  # set custom title
    plt.figure(figsize=(8,6))  # set figure size
    plt.imshow(wordcloud, interpolation="bilinear")  # set colour style
    plt.axis("off")  # disable axis
    plt.title(custom_title)  # set custom title
    plt.show()  # show plots
  
    

Plot input and output text length histogram. 

In [ ]:
df['input_length'] = df['input'].apply(lambda x: len(x.split(' '))) # Sum input text per row
df['output_length'] = df['output'].apply(lambda x: len(x.split(' ')))  # Sum output text per row
df_sum = df[['input_length', 'output_length']] # set temp df


fig, axs = plt.subplots(1,2,figsize=(14, 6)) # create figure and axis subplot objects

# Set common labels and titles
fig.suptitle("Input and Output Text Length Frequency")
fig.text(0.5, 0.04, 'Text Length', ha='center', va='center')
fig.text(0.06, 0.5, 'Frequency', ha='center', va='center', rotation='vertical')


df_sum.hist(ax=axs, bins=range(5, 60, 1)); # plot histograms


Set maximum text length to consider.

In [ ]:
MAX_LENGTH = 10  # Maximum text length to consider

Plot word frequency for input and output to determine word clipping value

In [ ]:
words = df['input'].str.split(expand=True).stack().value_counts() # list of word count
distribution = Counter(words) # count frequency in word count
plt.axis([0, 1400, 0, 10]) # x axis to maximum and y axis clipped to 10 frequency max so lower values are visibale
plt.title("Input Data Word Frequency Count") # set custom title
plt.grid(color='gray') # set grid colour
plt.bar(range(len(distribution)), distribution.values()); # plot bar




In [ ]:
words = df['output'].str.split(expand=True).stack().value_counts() # list of word count
distribution = Counter(words) # count frequency in word count
plt.axis([0, 1100, 0, 10]) # x axis to maximum and y axis clipped to 10 frequency max so lower values are visibale
plt.title("Output Data Word Frequency Count") # set custom title
plt.grid(color='gray') # set grid colour
plt.bar(range(len(distribution)), distribution.values()); # plot bar

Set minimum word count threshold for trimming.

In [ ]:
MIN_COUNT = 2   # Minimum word count threshold for trimming

### 4. Pre-Processing

Pre-processing code adapted from: https://pytorch.org/tutorials/beginner/chatbot_tutorial.html

#### 4.1 Vocabulary helper class

Sequences of words do not have an implicit mapping to a discrete numerical space. A vocabulary helper class is mapping each unique word in the dataset to an index value. The Voc class keeps a mapping from words to indexes, a reverse mapping of indexes to words, a count of each word and a total word count. The class provides methods for adding a word to the vocabulary (addWord), adding all words in a sentence (addSentence) and trimming infrequently seen words (trim). 

In [ ]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)
            

#### 4.2 Normalisation helper functions

Normalisation helper function to pre-process text:

1. Convert Unicode strings to ASCII using unicodeToAscii
2. Convert all letters to lowercase and trim all non-letter characters except for basic punctuation. Remove url, and words with .com and remove consecutive basic punctuation and keep one instance.
3. Filter out sentences with length greater than the MAX_LENGTH threshold (filterPairs) to facilitate training convergence.
4. main load prepare data function that is using the above.

In [ ]:


# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip()) # convert to lower and strip spaces on the start and end
    s = re.sub(r'[\?\.\!]+(?=[\?\.\!])', '', s) # remove basic punctuation multiple occurrences and keep one
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s) # remove all non-letter characters except for basic punctuation
    s = re.sub(r"http\S+", "", s) # remove words starting with http or https
    s = re.sub(r'\w+.com\s?','',s) # remove words ending with .com
    s = re.sub(r"\s+", r" ", s).strip() # remove multiple tabs and keep one, strip spaces on the start and end
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True if both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p, max_length):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < max_length and len(p[1].split(' ')) < max_length

# Filter pairs using filterPair condition
def filterPairs(pairs, max_length):
    return [pair for pair in pairs if filterPair(pair, max_length)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, max_length):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs, max_length)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

# Load/Assemble voc and pairs
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, MAX_LENGTH)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

For faster convergence during training, rare words are trimed out of the vocabulary using the *MIN_COUNT* threshold set on section **3.2 Data Analysis** . First call the *voc.trim* function and then filter out the pairs with trimmed words. Finally split data to train, validation, and test sets.

In [ ]:
def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index or len(word) < 1:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index or len(word) < 1:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)
for pair in pairs[:10]:
    print(pair)

Split dataset to Train, Validation, Test set.

In [ ]:
# Set ratio
tr_value = int(len(pairs) / 100 * 80)
val_value = int(len(pairs) / 100 * 90)

In [ ]:
tr_idx = slice(0,tr_value)
val_idx = slice(tr_value,val_value)
tst_idx = slice(val_value, -1)

# Slice to split
train_pairs = pairs[:][tr_idx]
validation_pairs = pairs[:][val_idx]
test_pairs = pairs[:][tst_idx]

print("Train Pairs: {:.0%} of total, Validation Pairs: {:.0%} of total, Test Pairs: {:.0%} of total".format(len(train_pairs) / len(pairs), len(validation_pairs) / len(pairs), len(test_pairs) / len(pairs)))

#### 4.3 Embedding, Padding, Mini-Batches

Using mini-batches heps to speed up training and also leverage GPU parallelization capabilities.

Using mini-batches also means that we must be mindful of the variation of sentence length in our batches. To accommodate sentences of different sizes in the same batch, we will make our batched input tensor of shape (max_length, batch_size), where sentences shorter than the max_length are zero padded after an EOS_token.

If we simply convert our English sentences to tensors by converting words to their indexes(indexesFromSentence) and zero-pad, our tensor would have shape (batch_size, max_length) and indexing the first dimension would return a full sequence across all time-steps. However, we need to be able to index our batch along time, and across all sequences in the batch. Therefore, we transpose our input batch shape to (max_length, batch_size), so that indexing across the first dimension returns a time step across all sentences in the batch. We handle this transpose implicitly in the zeroPadding function.


The inputVar function handles the process of converting sentences to tensor, ultimately creating a correctly shaped zero-padded tensor. It also returns a tensor of lengths for each of the sequences in the batch which will be passed to our decoder later.

The outputVar function performs a similar function to inputVar, but instead of returning a lengths tensor, it returns a binary mask tensor and a maximum target sentence length. The binary mask tensor has the same shape as the output target tensor, but every element that is a PAD_token is 0 and all others are 1.

batch2TrainData simply takes a bunch of pairs and returns the input and target tensors using the aforementioned functions.

In [ ]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

    

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len




# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)


### 5. Seq2Seq Model

The goal of a seq2seq model is to take a variable-length sequence as an input, and return a variable-length sequence as an output using a fixed-sized model.

Seq2seq code adapted from: https://pytorch.org/tutorials/beginner/chatbot_tutorial.html

#### 5.1 Encoder class

The encoder RNN iterates through the input sentence one token (e.g. word) at a time, at each time step outputting an “output” vector and a “hidden state” vector. The hidden state vector is then passed to the next time step, while the output vector is recorded. The encoder transforms the context it saw at each point in the sequence into a set of points in a high-dimensional space, which the decoder will use to generate a meaningful output for the given task. The encoder is using multi-layered Gated Recurrent Unit (GRU), invented by [Cho *et al.*](https://arxiv.org/pdf/1406.1078v3.pdf) (2014). We will use a bidirectional variant of the GRU.

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

#### 5.2 Luong attention class

[Luong *et al.*](https://arxiv.org/abs/1508.04025) (2015) improved upon [Bahdanau *et al.*](https://arxiv.org/abs/1409.0473) (2014) groundwork by creating “Global attention”. The key difference is that with “Global attention”, we consider all of the encoder’s hidden states, as opposed to [Bahdanau *et al.*](https://arxiv.org/abs/1409.0473) (2014) “Local attention”, which only considers the encoder’s hidden state from the current time step. Another difference is that with “Global attention”, we calculate attention weights, or energies, using the hidden state of the decoder from the current time step only. [Bahdanau *et al.*](https://arxiv.org/abs/1409.0473) (2014) attention calculation requires knowledge of the decoder’s state from the previous time step. Also, [Luong *et al.*](https://arxiv.org/abs/1508.04025) (2015) provides various methods to calculate the attention energies between the encoder output and decoder output which are called “score functions”.

In [ ]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

#### 5.3 Decoder class

The decoder RNN generates the response sentence in a token-by-token fashion. It uses the encoder’s context vectors, and internal hidden states to generate the next word in the sequence. It continues generating words until it outputs an EOS_token, representing the end of the sentence. For the decoder, we will manually feed our batch one time step at a time. This means that our embedded word tensor and GRU output will both have shape (1, batch_size, hidden_size).

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(DecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
        if self.attn_model != "none":
            self.attn = Attn(attn_model, hidden_size)   

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        rnn_output = rnn_output.squeeze(0)
        
        if self.attn_model != "none":
            # Calculate attention weights from the current GRU output
            attn_weights = self.attn(rnn_output, encoder_outputs)
            # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
            context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
            # Concatenate weighted context vector and GRU output using Luong eq. 5
            context = context.squeeze(1)
            concat_input = torch.cat((rnn_output, context), 1)
            concat_output = torch.tanh(self.concat(concat_input))
            # Predict next word using Luong eq. 6
            output = self.out(concat_output)
        else:
            output = self.out(rnn_output)

        output = F.softmax(output, dim=1)

        # Return output and final hidden state
        return output, hidden

### 6 Model Training Procedure

#### 6.1 Masked loss

Since we are dealing with batches of padded sequences, we cannot simply consider all elements of the tensor when calculating loss. We define maskNLLLoss to calculate our loss based on our decoder’s output tensor, the target tensor, and a binary mask tensor describing the padding of the target tensor. This loss function calculates the average negative log likelihood of the elements that correspond to a 1 in the mask tensor.

In [ ]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

#### 6.2 Train - Validation function

The train function contains the algorithm for a single training iteration (a single batch of inputs).

We will use a couple of improvement methods to aid in convergence:

The first method is using teacher forcing. This means that at some probability, set by teacher_forcing_ratio, we use the current target word as the decoder’s next input rather than using the decoder’s current guess. This technique acts as training wheels for the decoder, aiding in more efficient training. However, teacher forcing can lead to model instability during inference, as the decoder may not have a sufficient chance to truly craft its own output sequences during training. Thus, we must be mindful of how we are setting the teacher_forcing_ratio, and not be fooled by fast convergence.
The second method that we implement is gradient clipping. This is a commonly used technique for countering the “exploding gradient” problem. In essence, by clipping or thresholding gradients to a maximum value, we prevent the gradients from growing exponentially and either overflow (NaN), or overshoot steep cliffs in the cost function.

In [ ]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, teacher_forcing_ratio,max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]
    

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal


    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

Validation function

In [ ]:
def validate(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
     batch_size, max_length=MAX_LENGTH):
     

    # Set dropout layers to eval mode
    encoder.eval()
    decoder.eval()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]
    

    # Disable teacher forcing
    use_teacher_forcing = False

    # Forward batch of sequences through decoder one time step at a time

    for t in range(max_target_len):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden, encoder_outputs
        )
        # No teacher forcing: next input is decoder's own current output
        _, topi = decoder_output.topk(1)
        decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
        decoder_input = decoder_input.to(device)
        # Calculate and accumulate loss
        mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
        loss += mask_loss
        print_losses.append(mask_loss.item() * nTotal)
        n_totals += nTotal

    # Set dropout layers back to train mode
    encoder.train()
    decoder.train()    

    return sum(print_losses) / n_totals

#### 6.3 Training iterations

The trainIters function is responsible for running n_iterations of training given the passed models, optimizers, data, etc. This function is quite self explanatory, as we have done the heavy lifting with the train function.

In addition, we implement a validation loop and a loss plot method.

One thing to note is that when we save our model, we save a tarball containing the encoder and decoder state_dicts (parameters), the optimizers’ state_dicts, the loss, the iteration, etc. Saving the model in this way will give us the ultimate flexibility with the checkpoint. After loading a checkpoint, we will be able to use the model parameters to run inference, or we can continue training right where we left off.

In [ ]:
def showPlot(points, validation_points, model_name):
    plt.figure()
    fig, ax = plt.subplots(figsize=(14, 6))
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.5)
    ax.yaxis.set_major_locator(loc)
    ax.plot(points, color='r', label='Train Loss')
    ax.plot(validation_points, color='g', label='Validation Loss')
    # Adding legend, which helps us recognize the curve according to it's color, and custom title
    custom_title = model_name.replace("_", " ") + " Train - Validation Loss"
    ax.set_title(custom_title)
    ax.legend()
    

    # To load the display window
    plt.show()



def trainIters(model_name, voc, train_pairs, validation_pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, 
                decoder_n_layers, hidden_size, save_dir, n_iteration, batch_size, validate_every, save_every, clip, corpus_name, teacher_forcing_ratio):

    # Load batches for each  iteration
    training_batches = [batch2TrainData(voc, [random.choice(train_pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    validation_batches = [batch2TrainData(voc, [random.choice(validation_pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]                  

    # Initializations
    start_iteration = 1
    print_loss = 0
    valid_print_loss = 0
    plot_losses = []
    plot_valid_losses = []
    best_valid_loss = float('inf')
    valid_loss = 0

    # Training loop
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                   decoder, embedding, encoder_optimizer, decoder_optimizer,batch_size, clip, teacher_forcing_ratio)
        print_loss += loss
        
        if iteration % validate_every == 0:
            # Print progress
            print_loss_avg = print_loss / validate_every
            # Append loss to plot
            plot_losses.append(print_loss_avg)
            print_loss = 0

            validation_batch = validation_batches[iteration - 1]
            input_variable, lengths, target_variable, mask, max_target_len = validation_batch
            # Run a validation iteration with batch
            valid_loss = validate(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                        decoder, embedding, batch_size)
            
            plot_valid_losses.append(valid_loss)

            print("Iteration: {}; Percent complete: {:.1f}%".format(iteration, iteration / n_iteration * 100))
            print(f'\tTrain Loss: {print_loss_avg:.3f} | Validation Loss: {valid_loss:.3f}')

        # Save checkpoint
        if iteration % save_every == 0:
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))
  

    showPlot(plot_losses, plot_valid_losses, model_name)

#### 6.4 Greedy decoding

Greedy decoding is the decoding method that we use during training when we are NOT using teacher forcing. In other words, for each time step, we simply choose the word from decoder_output with the highest softmax value. This decoding method is optimal on a single time-step level.

In [ ]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:self.decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], dtype=torch.long)
        all_scores = torch.zeros([0])
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

### 7. Model Training

Model configuration variables and hyperparameters.

In [ ]:
# Configure models
model_names = ['RNN_With_Attention', 'RNN_Without_Attention']
attn_model = ['dot', 'none']
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

save_dir = os.path.join(data, "checkpoints")
save_dir

# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0005
decoder_learning_ratio = 5.0
n_iteration = 10000
validate_every = 1000
save_every = 5000



Train - Validation iterations loop

In [ ]:
idx = 0
for model_name in model_names:

    print('Initialising ' + model_name.replace("_", " ") + ' ...')


    print('Building encoder and decoder ...')
    # Initialize word embeddings
    embedding = nn.Embedding(voc.num_words, hidden_size)

    encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
    decoder = DecoderRNN(attn_model[idx], embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
    idx +=1
    # Use appropriate device
    encoder = encoder.to(device)
    decoder = decoder.to(device)

    # Ensure dropout layers are in train mode
    encoder.train()
    decoder.train()

    # Initialize optimizers
    print('Building optimizers ...')
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

    # If you have cuda, configure cuda to call
    for state in encoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    for state in decoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    # Run training iterations
    print("Started Training ...")
    trainIters(model_name, voc, train_pairs, validation_pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
            embedding, encoder_n_layers, decoder_n_layers, hidden_size, save_dir, n_iteration, batch_size,
            validate_every, save_every, clip, corpus_name, teacher_forcing_ratio)



### 8. Model Evaluation

Evaluate on text

Now that we have our decoding method defined, we can write functions for evaluating a string input sentence. The evaluate function manages the low-level process of handling the input sentence. We first format the sentence as an input batch of word indexes with batch_size==1. We do this by converting the words of the sentence to their corresponding indexes, and transposing the dimensions to prepare the tensor for our models. We also create a lengths tensor which contains the length of our input sentence. In this case, lengths is scalar because we are only evaluating one sentence at a time (batch_size==1). Next, we obtain the decoded response sentence tensor using our GreedySearchDecoder object (searcher). Finally, we convert the response’s indexes to words and return the list of decoded words.

In [ ]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to("cpu")
    lengths = lengths.to("cpu")
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words

Evaluate model function outputs input, target, actual prediction, bleu 1 and bleu 2 dataframe

In [ ]:
def evaluateModel(encoder, decoder, searcher, voc):
    input = []
    target = []
    predict = []
    b_1 = []
    b_2 = []
  
    for pair in test_pairs:
        # Get input sentence
        input_sentence = pair[0]
        # Evaluate sentence
        candidate = evaluate(encoder, decoder, searcher, voc, input_sentence)
        # Format and print response sentence
        candidate[:] = [x for x in candidate if not (x == 'EOS' or x == 'PAD')]
        reference = [pair[1].split(' ')]
        score_1 = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
        score_2 = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0))
        # save score to df
        input.append(pair[0])
        target.append(' '.join(map(str,reference[0])))
        predict.append(' '.join(map(str,candidate)))
        b_1.append(score_1)
        b_2.append(score_2)
        bleu_eval = pd.DataFrame({'Input sentence': input,'Target sentence': target,'Predicted sentence': predict,'BLEU-1 score': b_1,'BLEU-2 score': b_2})

    return bleu_eval

Once, the training is finished, we can now load the saved attention model and run evaluation.

In [ ]:
# Set checkpoint to load from; set to None if starting from scratch
loadFilename = True
checkpoint_iter = n_iteration # get the last checkpoint with attention
loadFilename = os.path.join(save_dir, model_names[0], corpus_name,
                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
                            '{}_checkpoint.tar'.format(checkpoint_iter))

# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']

if loadFilename:
    embedding.load_state_dict(embedding_sd)

if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)


Evaluate using test dataset and print results containing samples, BLEU score 1 and 2 and average BLEU 1 score

In [ ]:
# Set dropout layers to eval mode
encoder = encoder.to("cpu")
decoder = decoder.to("cpu")
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
bleu_score = evaluateModel(encoder, decoder, searcher, voc)
print("Average BLEU-1 score: ", bleu_score['BLEU-1 score'].mean())
print("Average BLEU-2 score: ", bleu_score['BLEU-2 score'].mean())
bleu_score.sort_values(by='BLEU-1 score', ascending=False).head(10)
